In [86]:
import warnings
import tensorflow as tf
import glob
from tqdm import tqdm
import midi_manipulation
import numpy as np
from tensorflow.python.ops import control_flow_ops
from distutils.version import LooseVersion
from utils import *

data_path = './data/edm_essential_melodies'

input_songs, target_songs = get_song_matrixes(data_path, 40, 20)

input_songs = np.array(input_songs)
target_songs = np.array(target_songs)


tokens = get_tokens(input_songs)
num_encoder_tokens = np.array(tokens).shape[0]
num_decoder_tokens = np.array(tokens).shape[0]


print('[*] Embedding Songs')
embeded_input_songs = get_embeded_songs(input_songs, tokens, num_encoder_tokens)
embeded_target_songs = embeded_input_songs

[*] Converting songs to matrix


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 88.90it/s]


[*] Converted 39 songs to matrix



(20, 156)
Processing song: 0/144
Processing song: 50/144
Processing song: 100/144
[*] Embedding Songs
Processing embed: 0/144
Processing embed: 50/144
Processing embed: 100/144


In [87]:
print(np.array(embeded_input_songs[0]).shape)


print(np.array(embed_song_to_song(embeded_input_songs[0], tokens)).shape)

# Finding the longest song in the dataset
max_encoder_seq_length = max([len(song) for song in embeded_input_songs])
max_decoder_seq_length = max([len(song) for song in embeded_target_songs])


print('Number of samples:', len(embeded_input_songs))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# Get input data in shape (num_sample, max_seq_length, num_tokens)
encoder_input_data, decoder_input_data, decoder_target_data = get_input_data(
                                                                        embeded_input_songs, 
                                                                        embeded_target_songs,
                                                                        max_encoder_seq_length, 
                                                                        num_encoder_tokens, 
                                                                        max_decoder_seq_length, 
                                                                        num_decoder_tokens)

(20, 839)
(20, 156)
Number of samples: 144
Number of unique input tokens: 839
Number of unique output tokens: 839
Max sequence length for inputs: 20
Max sequence length for outputs: 20
(144, 20, 839) (144, 20, 839)

Encoder input data shape: (144, 20, 839)
Decoder input data shape: (144, 20, 839)
Decoder target data shape: (144, 20, 839)



In [94]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Hyperparameters
batch_size  =   16      # Batch size for training.
epochs      =   120     # Number of epochs to train for.
latent_dim  =   512     # Latent dimensionality of the encoding space.
#num_samples =   10000   # Number of samples to train on.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print()
print(model.summary())
print()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, None, 839)    0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, None, 839)    0                                            
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, None, 512),  2768896     input_19[0][0]                   
__________________________________________________________________________________________________
lstm_12 (LSTM)                  [(None, None, 512),  2768896     input_20[0][0]                   
                                                                 lstm_11[0][1]                    
         

In [95]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy')

print('[*] Starting Training')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)
# model.load_weights('s2s'+data_path+'.h5')
print('[*] Ready to be used \n\n')

[*] Starting Training
Epoch 1/120
144/144 [==============================] - ETA: 16s - loss: 7.06 - ETA: 7s - loss: 7.0633 - ETA: 4s - loss: 7.059 - ETA: 2s - loss: 7.050 - ETA: 1s - loss: 7.039 - ETA: 1s - loss: 7.013 - ETA: 0s - loss: 6.939 - ETA: 0s - loss: 6.970 - 3s 18ms/step - loss: 6.8816
Epoch 2/120
144/144 [==============================] - ETA: 0s - loss: 6.325 - ETA: 0s - loss: 6.384 - ETA: 0s - loss: 6.360 - ETA: 0s - loss: 6.271 - ETA: 0s - loss: 6.273 - ETA: 0s - loss: 6.262 - ETA: 0s - loss: 6.270 - ETA: 0s - loss: 6.212 - 1s 4ms/step - loss: 6.1981
Epoch 3/120
144/144 [==============================] - ETA: 0s - loss: 6.103 - ETA: 0s - loss: 6.155 - ETA: 0s - loss: 5.998 - ETA: 0s - loss: 6.055 - ETA: 0s - loss: 6.001 - ETA: 0s - loss: 6.060 - ETA: 0s - loss: 6.038 - ETA: 0s - loss: 6.039 - 1s 4ms/step - loss: 6.0184
Epoch 4/120
144/144 [==============================] - ETA: 0s - loss: 6.188 - ETA: 0s - loss: 5.903 - ETA: 0s - loss: 6.069 - ETA: 0s - loss: 5.939 - ETA

144/144 [==============================] - ETA: 0s - loss: 3.397 - ETA: 0s - loss: 3.324 - ETA: 0s - loss: 3.179 - ETA: 0s - loss: 3.168 - ETA: 0s - loss: 3.066 - ETA: 0s - loss: 3.080 - ETA: 0s - loss: 3.082 - ETA: 0s - loss: 3.060 - 1s 4ms/step - loss: 3.0763
Epoch 31/120
144/144 [==============================] - ETA: 0s - loss: 2.885 - ETA: 0s - loss: 2.866 - ETA: 0s - loss: 2.872 - ETA: 0s - loss: 2.859 - ETA: 0s - loss: 2.859 - ETA: 0s - loss: 2.873 - ETA: 0s - loss: 2.867 - ETA: 0s - loss: 2.862 - 1s 4ms/step - loss: 2.8654
Epoch 32/120
144/144 [==============================] - ETA: 0s - loss: 2.926 - ETA: 0s - loss: 2.747 - ETA: 0s - loss: 2.732 - ETA: 0s - loss: 2.732 - ETA: 0s - loss: 2.732 - ETA: 0s - loss: 2.740 - ETA: 0s - loss: 2.724 - ETA: 0s - loss: 2.699 - 1s 4ms/step - loss: 2.6753
Epoch 33/120
144/144 [==============================] - ETA: 0s - loss: 2.416 - ETA: 0s - loss: 2.400 - ETA: 0s - loss: 2.395 - ETA: 0s - loss: 2.450 - ETA: 0s - loss: 2.463 - ETA: 0s - lo

144/144 [==============================] - ETA: 0s - loss: 0.604 - ETA: 0s - loss: 0.748 - ETA: 0s - loss: 0.747 - ETA: 0s - loss: 0.728 - ETA: 0s - loss: 0.787 - ETA: 0s - loss: 0.774 - ETA: 0s - loss: 0.754 - ETA: 0s - loss: 0.751 - 1s 4ms/step - loss: 0.7588
Epoch 60/120
144/144 [==============================] - ETA: 0s - loss: 0.761 - ETA: 0s - loss: 0.802 - ETA: 0s - loss: 0.805 - ETA: 0s - loss: 0.789 - ETA: 0s - loss: 0.733 - ETA: 0s - loss: 0.729 - ETA: 0s - loss: 0.710 - ETA: 0s - loss: 0.726 - 1s 4ms/step - loss: 0.7253
Epoch 61/120
144/144 [==============================] - ETA: 0s - loss: 0.681 - ETA: 0s - loss: 0.638 - ETA: 0s - loss: 0.702 - ETA: 0s - loss: 0.713 - ETA: 0s - loss: 0.699 - ETA: 0s - loss: 0.716 - ETA: 0s - loss: 0.719 - ETA: 0s - loss: 0.706 - 1s 4ms/step - loss: 0.7061
Epoch 62/120
144/144 [==============================] - ETA: 0s - loss: 0.694 - ETA: 0s - loss: 0.667 - ETA: 0s - loss: 0.610 - ETA: 0s - loss: 0.694 - ETA: 0s - loss: 0.699 - ETA: 0s - lo

144/144 [==============================] - ETA: 0s - loss: 0.279 - ETA: 0s - loss: 0.268 - ETA: 0s - loss: 0.302 - ETA: 0s - loss: 0.296 - ETA: 0s - loss: 0.303 - ETA: 0s - loss: 0.294 - ETA: 0s - loss: 0.284 - ETA: 0s - loss: 0.286 - 1s 4ms/step - loss: 0.2857
Epoch 89/120
144/144 [==============================] - ETA: 0s - loss: 0.330 - ETA: 0s - loss: 0.304 - ETA: 0s - loss: 0.295 - ETA: 0s - loss: 0.289 - ETA: 0s - loss: 0.321 - ETA: 0s - loss: 0.314 - ETA: 0s - loss: 0.304 - ETA: 0s - loss: 0.302 - 1s 4ms/step - loss: 0.3248
Epoch 90/120
144/144 [==============================] - ETA: 0s - loss: 0.338 - ETA: 0s - loss: 0.287 - ETA: 0s - loss: 0.272 - ETA: 0s - loss: 0.272 - ETA: 0s - loss: 0.279 - ETA: 0s - loss: 0.298 - ETA: 0s - loss: 0.313 - ETA: 0s - loss: 0.319 - 1s 4ms/step - loss: 0.3189
Epoch 91/120
144/144 [==============================] - ETA: 0s - loss: 0.291 - ETA: 0s - loss: 0.324 - ETA: 0s - loss: 0.389 - ETA: 0s - loss: 0.345 - ETA: 0s - loss: 0.326 - ETA: 0s - lo

144/144 [==============================] - ETA: 0s - loss: 0.129 - ETA: 0s - loss: 0.158 - ETA: 0s - loss: 0.153 - ETA: 0s - loss: 0.151 - ETA: 0s - loss: 0.152 - ETA: 0s - loss: 0.149 - ETA: 0s - loss: 0.151 - ETA: 0s - loss: 0.154 - 1s 4ms/step - loss: 0.1592
Epoch 118/120
144/144 [==============================] - ETA: 0s - loss: 0.131 - ETA: 0s - loss: 0.134 - ETA: 0s - loss: 0.137 - ETA: 0s - loss: 0.151 - ETA: 0s - loss: 0.151 - ETA: 0s - loss: 0.150 - ETA: 0s - loss: 0.146 - ETA: 0s - loss: 0.149 - 1s 4ms/step - loss: 0.1493
Epoch 119/120
144/144 [==============================] - ETA: 0s - loss: 0.126 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.127 - ETA: 0s - loss: 0.143 - ETA: 0s - loss: 0.137 - ETA: 0s - loss: 0.137 - ETA: 0s - loss: 0.138 - ETA: 0s - loss: 0.138 - 1s 4ms/step - loss: 0.1408
Epoch 120/120
144/144 [==============================] - ETA: 0s - loss: 0.139 - ETA: 0s - loss: 0.139 - ETA: 0s - loss: 0.133 - ETA: 0s - loss: 0.133 - ETA: 0s - loss: 0.143 - ETA: 0s -

In [96]:
# Save model
model.save('s2s'+'edm'+'.h5')

C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:2344: UserWarning: Layer lstm_12 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_11/while/Exit_2:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_11/while/Exit_3:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [97]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [98]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print(np.array(states_value).shape)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]

    return song_matrix


seq_length = max_encoder_seq_length

song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/edm_essential_melodies/Ralph Cowell EDM Essential Melodies MIDI Vol. 2 - 4.mid'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
print(np.array(encoder_input_data).shape)        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)

# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq = encoder_input_data1[:-1]

decoded_songs = []
decoded_songs.append(decode_sequence(input_seq))
for i in range(5):
    decoded_songs.append(decode_sequence((
                    np.reshape(decoded_songs[-1], 
                        (1, decoded_songs[-1].shape[0], decoded_songs[-1].shape[1])))))


decoded_song = embed_song_to_song(np.concatenate(decoded_songs, axis = 0), tokens)

# Converting Song to midi from matrix
print('[*] Converting and saving song')
midi_manipulation.noteStateMatrixToMidi(decoded_song, 'edm_example_40songs_20seqlength.midi')

(3, 20, 156)
Processing embed: 0/3
[*] Encoding-Decoding
(2, 2, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
[*] Converting and saving song


In [15]:
np.sum(decoded_song)

2937

In [134]:
def mixing_sequence(input_seq1, input_seq2):
    # Encode the input as state vectors.
    states_value2 = encoder_model.predict(input_seq1)
    states_value = encoder_model.predict(input_seq2)
    
    for i, states1 in enumerate(states_value):
        for j, states2 in enumerate(states1):
            for k, states3 in enumerate(states2):
                states_value[i][j][k] = (states3*1.5 + states_value2[i][j][k]*2)/2

    #print(type(states_value))
    #states_value[:,:2,:] *= np.array(states_value2)
    #print(np.array(states_value1).shape)
    #for i, (state1, state2) in enumerate(zip(states_value1, states_value2)):
    #    for j, (s2_1, s2_2) in enumerate(zip(state1, state2)):
     #       id1 = np.argmax(s2_1)
    #        id2 = np.argmax(s2_2)
     #       idx = (id1+id2) // 2
     #       states_value[i, j, idx] = 1
      #      break

        
            
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]


    return song_matrix

seq_length = max_encoder_seq_length

song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/edm_essential_melodies/Ralph Cowell EDM Essential Melodies MIDI Vol. 1 - 6.mid'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)

# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq1 = encoder_input_data1[:-1]


song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/edm_essential_melodies/Ralph Cowell EDM Essential Melodies MIDI Vol. 2 - 10.mid'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)
    
# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq2 = encoder_input_data1[:-1]

decoded_songs = []
decoded_songs.append(mixing_sequence(input_seq1, input_seq2))
for i in range(2):
    decoded_songs.append(decode_sequence((
                    np.reshape(decoded_songs[-1], 
                        (1, decoded_songs[-1].shape[0], decoded_songs[-1].shape[1])))))


decoded_song = embed_song_to_song(np.concatenate(decoded_songs, axis = 0), tokens)
edm+=1
# Converting Song to midi from matrix
print('[*] Converting and saving song')
midi_manipulation.noteStateMatrixToMidi(decoded_song, 'mixing_edm'+str(edm))

Processing embed: 0/3
[*] Encoding-Decoding
Processing embed: 0/3
[*] Encoding-Decoding
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
[*] Converting and saving song


In [ ]:
np.zeros((1,1,1,11432)) + np.zeros((2,1,1024)).tolist()